In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import *
from scipy.optimize import linprog

## Scratch Work

#### the main code is at the bottom

In [3]:
def nonzero_arr(J,alpha):
    np.set_printoptions(formatter={'float': lambda x: "{:.2f}".format(x)})
    arr = np.full((1, J), 1 / (J * (1 - alpha)))
    flat_arr = np.squeeze(arr)
    rounded_arr = np.round(flat_arr, 2)
    print(np.array2string(rounded_arr, separator=', '))


In [12]:
def first_constraint(J, n, alpha):
    nonzero_arr = np.full((1, J+1), 1 / (J * (1 - alpha)))
    # nonzero_arr = np.round(np.squeeze(nonzero_arr))
    nonzero_arr[-1,-1] = 1
    zero_arr = np.zeros((1,n))
    return np.concatenate((zero_arr, nonzero_arr), axis=1)
first_constraint(5,5,0.99)

array([[ 0.,  0.,  0.,  0.,  0., 20., 20., 20., 20., 20.,  1.]])

In [ ]:
# def create_named_matrix(rows, cols, prefix):
#     matrix = np.zeros((rows, cols + 1), dtype=object)
#     for i in range(rows):
#         for j in range(cols):
#             name = prefix + str(i+1) + str(j+1)
#             matrix[i, j] = '-' + name
#         matrix[i, -1] = -1
#     return matrix

# create_named_matrix(3,5,'y')


In [13]:
def create_matrix(J, n, data=None):
    
    matrix = np.zeros((J, n))
    # matrix[:, -1] = -1
    if data is not None:
        matrix[:data.shape[0], :data.shape[1]] = data * -1
    return matrix

create_matrix(3,4, data=np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]))

array([[ -1.,  -2.,  -3.,  -4.],
       [ -5.,  -6.,  -7.,  -8.],
       [ -9., -10., -11., -12.]])

In [ ]:
# gets the mean value of the column vector (y11, y12, ...)
# def meanValofYs(J, n, data=None):
#     matrix = np.zeros((n, n+1))
#     matrix[:, -1] = -1
#     if data is not None:
#         matrix[:data.shape[0], :data.shape[1]] = data
    
#     def column_averages(matrix):
#         averages = []
#         for i in range(matrix.shape[1]-1):
#             col_sum = np.sum(matrix[:, i])
#             col_avg = col_sum / matrix.shape[0]
#             averages.append(col_avg)
#         return averages
    
#     return column_averages(matrix)

# meanValofYs(3,4,data=np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]))
# meanValofYs(6,3,None)

In [14]:
def objective_func(J, n, data=None):
    matrix = np.zeros((J, n+1))
    matrix[:, -1] = -1
    if data is not None:
        matrix[:data.shape[0], :data.shape[1]] = data
    
    def column_averages(matrix):
        averages = []
        for i in range(matrix.shape[1]-1):
            col_sum = np.sum(matrix[:, i])
            col_avg = col_sum / matrix.shape[0]
            averages.append(col_avg * -1)  # multiply each average by -1
        return averages
    
    return column_averages(matrix) + [0]*(J+1)

objective_func(6,3,None)

[-0.0, -0.0, -0.0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
def negID_matrix(J):
    matrix = np.zeros((J, J+1), dtype=int)
    matrix[:, -1] = -1
    for i in range(J):
        if i < J:
            matrix[i, i] = -1
    return matrix

negID_matrix(3)

array([[-1,  0,  0, -1],
       [ 0, -1,  0, -1],
       [ 0,  0, -1, -1]])

In [16]:
def second_constraint(J,n,data):
    A = create_matrix(J, n, data)
    B = negID_matrix(J)  
    C = np.concatenate((A,B), axis=1)
    return C

second_constraint(5,3,None)

array([[ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.]])

In [ ]:
first_constraint(5,3,0.95)

## matrix A

In [21]:
def matrix_A(J, n, alpha, data):
    first = first_constraint(J, n, alpha)
    second = second_constraint(J, n, data)

    # return first, second
    # return np.concatenate((np.array([first]), second))
    return np.concatenate((first, second))

matrix_A(5,3,0.95,None)

array([[ 0.,  0.,  0.,  4.,  4.,  4.,  4.,  4.,  1.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.]])

=================================================================

In [22]:
def average_up_to_n(A, n):
    return np.mean(A[:, :n], axis=0)

A = matrix_A(6, 3, 0.95, None)
average_cols = average_up_to_n(A, 3)
print(average_cols)

[0. 0. 0.]


In [23]:
from scipy.optimize import linprog

In [24]:
#these are the functions for the column matrix 'b' in the inequality Ax <= b
def b1(n, omega):
    q = np.ones(n) # FIXME
    x0 = np.ones(n) # FIXME
    # print(q, x0)
    return omega*sum(q[i]*x0[i] for i in range(n))

def b2(n):
    q = np.ones(n) # FIXME
    x0 = np.ones(n) # FIXME
    # print(q, x0)
    return -1* sum(q[i]*x0[i] for i in range(n))

b1(10,3), b2(10)



(30.0, -10.0)

In [25]:
# column matrix 'b' in the inequality Ax <= b
def column_b(J,n, omega):
    first = np.array([b1(n, omega)])
    rest = np.array([b2(n) for i in range(J)])
    return np.concatenate((first, rest), axis=None)

column_b(6,10,0.5)



array([  5., -10., -10., -10., -10., -10., -10.])

In [42]:
c = np.array(objective_func(10000,30,data=ran)) # 10 variables
A = matrix_A(10000,30,0.99, data=ran)
b = column_b(10000,30,0.5)
aeq = np.array([[236.394, 282.302, 257.257,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ]])
beq = 24226.612170246597

# np.set_printoptions(threshold=np.inf)
c,A,b,aeq,beq

# res = linprog(c=c, A_ub =A, b_ub =b, A_eq = aeq, b_eq = beq, bounds=[(0,None)]*(10000+30+1))

# res.message

(array([-5.012, -5.01 , -5.036, ...,  0.   ,  0.   ,  0.   ]),
 array([[ 0.   ,  0.   ,  0.   , ...,  0.01 ,  0.01 ,  1.   ],
        [-2.37 , -0.509, -9.587, ...,  0.   ,  0.   , -1.   ],
        [-9.767, -3.46 , -3.797, ...,  0.   ,  0.   , -1.   ],
        ...,
        [-2.165, -3.529, -1.071, ...,  0.   ,  0.   , -1.   ],
        [-8.884, -5.448, -1.987, ..., -1.   ,  0.   , -1.   ],
        [-3.414, -7.902, -8.876, ...,  0.   , -1.   , -1.   ]]),
 array([ 15., -30., -30., ..., -30., -30., -30.]),
 array([[236.394, 282.302, 257.257,   0.   ,   0.   ,   0.   ,   0.   ,
           0.   ,   0.   ]]),
 24226.612170246597)

# =======================================

In [46]:
def equality_equ(J,n):
    x0 = np.random.uniform(0,300, size=n) # initial stock values
    q = np.random.uniform(0,100, size=n) # stock share

    A_eq = np.array([[x0[i] if i<n else 0 for i in range(n+J+1)]])
    b_eq = sum(q[i]*x0[i] for i in range(n))
    return A_eq, b_eq

equality_equ(10000,30)

(array([[234.394,  89.358,  57.423, ...,   0.   ,   0.   ,   0.   ]]),
 238009.06254030933)

In [94]:
# arbitrarily made matrix for number of stocks with scenarios
ran = np.random.uniform(0,10, size=(10000,30))
np.set_printoptions(precision= 3, suppress=True)

ran, ran.shape

(array([[4.818, 8.513, 8.649, ..., 3.205, 0.943, 2.969],
        [1.036, 0.689, 6.185, ..., 5.056, 0.79 , 3.402],
        [8.689, 5.655, 7.704, ..., 1.766, 3.149, 2.732],
        ...,
        [4.378, 7.259, 9.873, ..., 5.238, 9.358, 7.893],
        [9.318, 0.234, 3.708, ..., 3.417, 7.734, 5.914],
        [9.15 , 6.748, 8.825, ..., 7.256, 6.936, 6.043]]),
 (10000, 30))

# =================================================================================

In [91]:

def main(J,n, alpha, omega, data):

    def nonzero_arr(J,alpha):
        np.set_printoptions(formatter={'float': lambda x: "{:.2f}".format(x)})
        arr = np.full((1, J), 1 / (J * (1 - alpha)))
        flat_arr = np.squeeze(arr)
        rounded_arr = np.round(flat_arr, 2)
        print(np.array2string(rounded_arr, separator=', '))

    def first_constraint(J, n, alpha):
        nonzero_arr = np.full((1, J+1), 1 / (J * (1 - alpha)))
        # nonzero_arr = np.round(np.squeeze(nonzero_arr))
        nonzero_arr[-1,-1] = 1
        zero_arr = np.zeros((1,n))
        return np.concatenate((zero_arr, nonzero_arr), axis=1)
    
    def create_matrix(J, n, data=None):
        matrix = np.zeros((J, n))
        # matrix[:, -1] = -1
        if data is not None:
            matrix[:data.shape[0], :data.shape[1]] = data * -1
        return matrix

    def objective_func(J, n, data=None):
        matrix = np.zeros((J, n+1))
        matrix[:, -1] = -1
        if data is not None:
            matrix[:data.shape[0], :data.shape[1]] = data
        
        def column_averages(matrix):
            averages = []
            for i in range(matrix.shape[1]-1):
                col_sum = np.sum(matrix[:, i])
                col_avg = col_sum / matrix.shape[0]
                averages.append(col_avg * -1)  # multiply each average by -1
            return averages
        
        return column_averages(matrix) + [0]*(J+1)
    
    def negID_matrix(J):
        matrix = np.zeros((J, J+1), dtype=int)
        matrix[:, -1] = -1
        for i in range(J):
            if i < J:
                matrix[i, i] = -1
        return matrix
    
    def second_constraint(J,n,data):
        A = create_matrix(J, n, data)
        B = negID_matrix(J)  
        C = np.concatenate((A,B), axis=1)
        return C
    
    def matrix_A(J, n, alpha, data):
        first = first_constraint(J, n, alpha)
        second = second_constraint(J, n, data)

        # return first, second
        # return np.concatenate((np.array([first]), second))
        return np.concatenate((first, second))
    
    def average_up_to_n(A, n):
        return np.mean(A[:, :n], axis=0)
    
    def b1(n, omega):
        q = np.ones(n) # FIXME
        x0 = np.ones(n) # FIXME
        # print(q, x0)
        return omega*sum(q[i]*x0[i] for i in range(n))

    def b2(n):
        q = np.ones(n) # FIXME
        x0 = np.ones(n) # FIXME
        # print(q, x0)
        return -1* sum(q[i]*x0[i] for i in range(n))
    
    def column_b(J,n, omega):
        first = np.array([b1(n, omega)])
        rest = np.array([b2(n) for i in range(J)])
        return np.concatenate((first, rest), axis=None)
    

    x0 = np.random.uniform(0,300, size=n)
    q = np.random.uniform(0,100, size=n)

    c = np.array(objective_func(J,n,data)) # objective function
    A = matrix_A(J,n,alpha, data)   # matrix A
    b = column_b(J,n,omega)     # constraint of matrix A
    aeq = np.array([[x0[i] if i<n else 0 for i in range(n+J+1)]])
    beq = sum(q[i]*x0[i] for i in range(n))

    res = linprog(c=c, A_ub =A, b_ub =b, A_eq = aeq, b_eq = beq, bounds=[(0,None)]*(J+n+1))





    return x0,q,c,A,b,aeq,beq, res.fun, res




In [108]:
# this is more human-friednly structured than 'main'
# This is the actual function that I used to carry on.


def main2(J,n, alpha, omega, data):
    '''
    There are mainly 3 functions in main2:
        1. function that creates the objective function
        2. function that creates matrix A 
        3. function that creates the column vector b
    
    '''

    def objective_func(J, n, data=None): # this is the objective function
        matrix = np.zeros((J, n+1))
        matrix[:, -1] = -1
        if data is not None:
            matrix[:data.shape[0], :data.shape[1]] = data
        
        def column_averages(matrix):
            averages = []
            for i in range(matrix.shape[1]-1):
                col_sum = np.sum(matrix[:, i])
                col_avg = col_sum / matrix.shape[0]
                averages.append(col_avg * -1)  # multiply each average by -1
            return averages
        
        return column_averages(matrix) + [0]*(J+1)
    

    def matrix_A(J, n, alpha, data): # this defines the A matrix

        def first_constraint(J, n, alpha):

            def nonzero_arr(J,alpha):
                np.set_printoptions(formatter={'float': lambda x: "{:.2f}".format(x)})
                arr = np.full((1, J), 1 / (J * (1 - alpha)))
                flat_arr = np.squeeze(arr)
                rounded_arr = np.round(flat_arr, 2)
                print(np.array2string(rounded_arr, separator=', '))

            nonzero_arr = np.full((1, J+1), 1 / (J * (1 - alpha)))
            # nonzero_arr = np.round(np.squeeze(nonzero_arr))
            nonzero_arr[-1,-1] = 1
            zero_arr = np.zeros((1,n))
            return np.concatenate((zero_arr, nonzero_arr), axis=1)

        def second_constraint(J,n,data):

            def create_matrix(J, n, data=None):
                matrix = np.zeros((J, n))
                # matrix[:, -1] = -1
                if data is not None:
                    matrix[:data.shape[0], :data.shape[1]] = data * -1
                return matrix
            
            def negID_matrix(J):
                matrix = np.zeros((J, J+1), dtype=int)
                matrix[:, -1] = -1
                for i in range(J):
                    if i < J:
                        matrix[i, i] = -1
                return matrix
            
            A = create_matrix(J, n, data)
            B = negID_matrix(J)  
            C = np.concatenate((A,B), axis=1)
            return C
    
        first = first_constraint(J, n, alpha)
        second = second_constraint(J, n, data)

        # return first, second
        # return np.concatenate((np.array([first]), second))
        return np.concatenate((first, second))
    
    
    def column_b(J,n, omega): # this defines column vector b on the right side of the inequality

        def b1(n, omega):
                q = np.ones(n) # FIXME
                x0 = np.ones(n) # FIXME
                # print(q, x0)
                return omega*sum(q[i]*x0[i] for i in range(n))

        def b2(n):
            q = np.ones(n) # FIXME
            x0 = np.ones(n) # FIXME
            # print(q, x0)
            return -1* sum(q[i]*x0[i] for i in range(n))

        first = np.array([b1(n, omega)])
        rest = np.array([b2(n) for i in range(J)])
        return np.concatenate((first, rest), axis=None)


    x0 = np.random.uniform(0,300, size=n) #this is supposed to be the initial number of stocks
    q = np.random.uniform(0,100, size=n)

    c = np.array(objective_func(J,n,data)) # objective function
    A = matrix_A(J,n,alpha, data)   # matrix A
    b = column_b(J,n,omega)     # constraint of matrix A
    aeq = np.array([[x0[i] if i<n else 0 for i in range(n+J+1)]])
    beq = sum(q[i]*x0[i] for i in range(n))

    res = linprog(c=c, A_ub =A, b_ub =b, A_eq = aeq, b_eq = beq, bounds=[(0,None)]*(J+n+1))





    return res.fun, res

In [102]:
main2(10000,30,0.99, 0.01, data=ran)

(-26773.744941787205,
            con: array([0.])
  crossover_nit: 0
          eqlin:  marginals: array([-0.105])
   residual: array([0.])
            fun: -26773.744941787205
        ineqlin:  marginals: array([-2.303, -0.   , -0.   , ..., -0.   , -0.   , -0.   ])
   residual: array([    0.   ,  8805.993,  5436.262, ..., 34298.565, 46354.227,
        16172.23 ])
          lower:  marginals: array([13.787, 19.934, 18.698, ...,  0.023,  0.023,  2.234])
   residual: array([0., 0., 0., ..., 0., 0., 0.])
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
            nit: 71
          slack: array([    0.   ,  8805.993,  5436.262, ..., 34298.565, 46354.227,
        16172.23 ])
         status: 0
        success: True
          upper:  marginals: array([0., 0., 0., ..., 0., 0., 0.])
   residual: array([inf, inf, inf, ..., inf, inf, inf])
              x: array([0., 0., 0., ..., 0., 0., 0.]))